In [1]:
import cv2
import mediapipe as mp
import torch
import numpy as np
import pandas as pd
from CNNModel import CNNModel

In [2]:
# Load the model
model = CNNModel()
model.load_state_dict(torch.load("CNN_model_alphabet_SIBI.pth"))

C:\Users\ApriZon\AppData\Local\Temp\ipykernel_8416\1359024087.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("CNN_model_alphabet_SIBI.p

<All keys matched successfully>

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
# Initialize MediaPipe Hands and drawing utilities
handTracker = mp.solutions.hands
handDetector = handTracker.Hands(static_image_mode=True, min_detection_confidence=0.2)
drawing = mp.solutions.drawing_utils
drawingStyles = mp.solutions.drawing_styles

In [5]:
# Class mapping
classes = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
    'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19,
    'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25
}

In [6]:
model.eval()

CNNModel(
  (cnnLayers): Sequential(
    (0): Conv1d(63, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(2,))
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(2,))
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Dropout(p=0.3, inplace=False)
    (11): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(2,))
    (12): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv1d(256, 512, kernel_size=(5,), strid

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from webcam.")
        break

    # Flip the frame for a mirrored view
    frame = cv2.flip(frame, 1)

    height, width, _ = frame.shape
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgMediapipe = handDetector.process(frameRGB)

    coordinates = []
    x_Coordinates = []
    y_Coordinates = []
    z_Coordinates = []

    if imgMediapipe.multi_hand_landmarks:
        for handLandmarks in imgMediapipe.multi_hand_landmarks:
            drawing.draw_landmarks(
                frame,  # Image to draw
                handLandmarks,  # Model output
                handTracker.HAND_CONNECTIONS,  # Hand connections
                drawingStyles.get_default_hand_landmarks_style(),
                drawingStyles.get_default_hand_connections_style()
            )

            data = {}
            for lm in handLandmarks.landmark:
                x_Coordinates.append(lm.x)
                y_Coordinates.append(lm.y)
                z_Coordinates.append(lm.z)

            for i, landmark in enumerate(handTracker.HandLandmark):  # Apply Min-Max normalization
                lm = handLandmarks.landmark[i]
                data[f'{landmark.name}_x'] = lm.x - min(x_Coordinates)
                data[f'{landmark.name}_y'] = lm.y - min(y_Coordinates)
                data[f'{landmark.name}_z'] = lm.z - min(z_Coordinates)

            coordinates.append(data)

        x1 = int(min(x_Coordinates) * width) - 10
        y1 = int(min(y_Coordinates) * height) - 10
        x2 = int(max(x_Coordinates) * width) - 10
        y2 = int(max(y_Coordinates) * height) - 10

        predictions = []
        coordinates = pd.DataFrame(coordinates)
        coordinates = np.reshape(coordinates.values, (coordinates.shape[0], 63, 1))
        coordinates = torch.from_numpy(coordinates).float()

        with torch.no_grad():
            outputs = model(coordinates)
            _, predicted = torch.max(outputs.data, 1)
            predictions = predicted.cpu().numpy()

        predicted_character = [key for key, value in classes.items() if value == predictions[0]][0]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(
            frame, predicted_character, (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA
        )

    cv2.imshow('Hand Gesture Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit the loop if 'q' is pressed

In [ ]:
cap.release()
cv2.destroyAllWindows()